In [1]:
import os
import pandas as pd
import re
import numpy as np

In [2]:
# read all the crawled data in the same directory
os.chdir("D:\Projects\LEGO\LEGO_DATA2")
files = os.listdir()

In [3]:
# transfer into dataframes
df_lst = [pd.read_csv(file) for file in files]
filenames = [file.split(".")[0] for file in files]

In [4]:
df_dic = dict(zip(filenames,df_lst))

In [5]:
def price_us(txt):
    
    lst = txt.split(",")
    price = "missing"
    for item in lst:
        if '$' in item:
            price = float(item.replace("$","").replace("|","").strip())
            break
        elif "€" in item:
            price = round(float(item.replace("€","").replace("|","").strip())*1.13,2)
            break
    return price

In [6]:
# clean datasets with the same structure
def clean_data(file):
    df = df_dic[file]
    
    # delete NA in price, set_ID
    df = df[pd.notnull(df['price'])]
    df = df[pd.notnull(df['set_ID'])]
    
    # transform price(keep forms contain $ or €) to a new column named price_USD
    df['price_USD'] = df['price'].apply(price_us)
    
    # transform rating into float number
    df['rating'] = pd.to_numeric(df['rating'])
    
    # create a new column called year
    df['year'] = int(file)
    
    # delete ":" in set_ID
    df['set_ID'] = df['set_ID'].str.replace(":","")
    df['set_ID'] = df['set_ID'].str.strip()
    
    return df

In [26]:
# iterate over all dataframes
df_clean = [clean_data(file) for file in filenames]
data = pd.concat(df_clean)
print(data.shape)

(9042, 6)


In [27]:
data.head()

,name,price,set_ID,rating,price_USD,year
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal",215,4.0,0.5,1961
2,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal",217,4.0,0.5,1961
3,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal",218,4.0,0.5,1961
4,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal",219,4.0,0.5,1961
5,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal",220,4.0,0.5,1961


In [28]:
data = data.reset_index(drop=True)

In [29]:
data.set_ID[0]

'215'

In [30]:
data.head()

,name,price,set_ID,rating,price_USD,year
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal",215,4.0,0.5,1961
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal",217,4.0,0.5,1961
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal",218,4.0,0.5,1961
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal",219,4.0,0.5,1961
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal",220,4.0,0.5,1961


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9042 entries, 0 to 9041
Data columns (total 6 columns):
name         9042 non-null object
price        9042 non-null object
set_ID       9042 non-null object
rating       8852 non-null float64
price_USD    9042 non-null float64
year         9042 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 423.9+ KB


In [32]:
data[data.name == "Gears"]

,name,price,set_ID,rating,price_USD,year
45,Gears,"43,$4.95 ,11.5c,Box,Retail,Normal",001,NaN,4.95,1965


In [33]:
data.set_ID = data.set_ID.apply('"{}"'.format)

In [34]:
data[data.name == "Gears"]

,name,price,set_ID,rating,price_USD,year
45,Gears,"43,$4.95 ,11.5c,Box,Retail,Normal","""001""",NaN,4.95,1965


In [36]:
data.to_csv("D:\Projects\LEGO\Rebrickable\price.csv",index=False)